In [20]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
import os
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


def get_model(model: str = 'deepseek-r1:7b', provider: str = 'local'):
    if (provider == 'local'):
        from langchain_ollama import ChatOllama
        llm = ChatOllama(model=model, temperature=0)
        return llm
    elif (provider == 'llama'):
        # supports tools calling
        from langchain_ollama import ChatOllama
        llm = ChatOllama(model='llama3.1', temperature=0)
        return llm
    elif (provider == 'aws'):
        from langchain_aws import ChatBedrockConverse
        import boto3
        access_key = os.getenv('ACCESS_KEY')
        secret_key = os.getenv('SECRET_KEY')
        bedrock_client = boto3.client('bedrock-runtime',
                                      region_name='us-east-1',
                                      aws_access_key_id=access_key,
                                      aws_secret_access_key=secret_key)
        llm = ChatBedrockConverse(client=bedrock_client,
                                  model=model,
                                  temperature=0)
        return llm


def get_embeddings(model:str='deepseek-r1:7b', provider:str='local'):
    if(provider == 'local'):
        from langchain_ollama import OllamaEmbeddings
        embeddings=OllamaEmbeddings(model=model)
        return embeddings
    elif(provider == 'aws'):
        from langchain_aws import BedrockEmbeddings
        import boto3
        access_key=os.getenv('ACCESS_KEY')
        secret_key=os.getenv('SECRET_KEY')
        bedrock_client=boto3.client('bedrock-runtime', region_name='us-east-1', aws_access_key_id=access_key, aws_secret_access_key=secret_key)
        embeddings=BedrockEmbeddings(client=bedrock_client, model_id=model)
        return embeddings


workflow = StateGraph(state_schema=MessagesState)

model=get_model(provider='llama')


# Define the function that calls the model
def call_model(state: MessagesState):
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability."
    )
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = model.invoke(messages)
    return {"messages": response}


# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add simple in-memory checkpointer
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [21]:
app.invoke(
    {"messages": [HumanMessage(content="Translate to French: I love programming.")]},
    config={"configurable": {"thread_id": "1"}},
)


{'messages': [HumanMessage(content='Translate to French: I love programming.', additional_kwargs={}, response_metadata={}, id='fa4bd567-f31a-4d15-b3df-0b3f7a0b0c58'),
  AIMessage(content='The translation is:\n\n"J\'adore le programmation."\n\nHowever, if you want to be more idiomatic and use a more common expression in French, you could say:\n\n"Je suis passionné(e) par la programmation." (If you\'re male)\n"Je suis passionnée par la programmation." (If you\'re female)\n\nOr simply:\n"J\'aime programmer."\n\nAll of these expressions convey the same idea: that you enjoy programming.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-03-21T11:40:55.577499Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2655982208, 'load_duration': 33982250, 'prompt_eval_count': 39, 'prompt_eval_duration': 148618583, 'eval_count': 95, 'eval_duration': 2472230625, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-9a97310b-8ac

In [22]:
app.invoke(
    {"messages": [HumanMessage(content="What did I just ask you?")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content='Translate to French: I love programming.', additional_kwargs={}, response_metadata={}, id='fa4bd567-f31a-4d15-b3df-0b3f7a0b0c58'),
  AIMessage(content='The translation is:\n\n"J\'adore le programmation."\n\nHowever, if you want to be more idiomatic and use a more common expression in French, you could say:\n\n"Je suis passionné(e) par la programmation." (If you\'re male)\n"Je suis passionnée par la programmation." (If you\'re female)\n\nOr simply:\n"J\'aime programmer."\n\nAll of these expressions convey the same idea: that you enjoy programming.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-03-21T11:40:55.577499Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2655982208, 'load_duration': 33982250, 'prompt_eval_count': 39, 'prompt_eval_duration': 148618583, 'eval_count': 95, 'eval_duration': 2472230625, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': None}}, id='run-9a97310